# Lab Assignment One: Exploring Table Data
Arely Alcantara, Emily Fashenpour

## 1. Business Understanding

Do you ever stop to think about your food and any diseases that it might carry? Wouldn't it be great to have a resource to see what foods to avoid to stay healthy? One in six people in the United States get sick for eating contaminated food per year - how crazy is that??

The Foodborne Disease Outbreaks dataset includes 19119 reported outbreaks from the years 1998 to 2015. This data was reported to the Center of Disease Control and Prevention, and it lists the location, date, and food of the incident - just to name a few features. This dataset can be found and downloaded from kaggle.com

We believe that this data was collected in order to know what food or places to stay away from, or research a little more before eating. We hope that by analyzing this dataset, and digging deeper into what its features entail, could greatly benefit many. Third parties might be interested in sharing this information with the general public in order to prevent any future deaths or hospitalizations. A new food business could also leverage this information and use it to promote/advertise how their mission will be to provide quality and healthy food.

Our findings could save lives, and help people make smarter food choices. We can share our knowledge with others and hope to make a difference. Eating is fun, but not when your health is at risk. Smarter choices, better food.

Dataset URL: https://www.kaggle.com/cdc/foodborne-diseases

Question of Interest: Which foods are most likely to lead to hospitalization, or essentially death? In other words, what foods/places should one avoid?

## 2. Data Understanding

### 2.1 Data Defining & Description

In [6]:
import pandas as pd

#read data from csv using pandas
df = pd.read_csv('data/outbreaks.csv')



#how do we want to define data types? 
''' 
ordinal -> integers
continuous -> floats 
categorical -> nothing
look at second notebook for examples
'''

' \nordinal -> integers\ncontinuous -> floats \ncategorical -> nothing\nlook at second notebook for examples\n'

discuss attributes collected

### 2.2 Data Quality

explain missing data, or duplicate data. 

visualize entries that are missing (mistakes? why does these quality issues exist?)

how to deal with these issues? 


CHOOSE: elimination or imputation and JUSTIFY

## 3. Data Visualization

### 3.1 Data Exploration

CHOOSE and VISUALIZE distributions for a subset of single attributes


use histograms, kernel density estimation, box plots, etc. 


describe anything meaningful found from the visualizations


**CAN USE other sourses to boost visualizations

VISUALIZE at least 5 attributes (one categorical, one numerical)

In [ ]:
Visualize Relationships between a subset of attributes.

use visualization method


explain any interesting relationships

visualize at least 3 subsets